In [1]:
import duckdb
import os
from dotenv import load_dotenv
import duckdb

load_dotenv()  # loads .env into environment

db_path = os.getenv("DUCKDB_PATH", "data.duckdb")  # default fallback

con = duckdb.connect(db_path)  # this creates/opens the file

In [2]:
import pandas as pd
import duckdb
from pathlib import Path

# Path to the Excel file
EXCEL_PATH = Path(
    "/home/ettore/projects/hackathons/MVA_hackathon_2025/vintage_ai/data/raw/asset_classic_new_data.xlsx")

# --- Helper to load and normalize a sheet --- #
def load_and_melt(sheet_name: str, metric_name: str) -> pd.DataFrame:
    df = pd.read_excel(EXCEL_PATH, sheet_name=sheet_name)
    df.rename(columns={df.columns[0]: "ts"}, inplace=True)  # rename first column to 'ts'
    df["ts"] = pd.to_datetime(df["ts"], errors="coerce")    # coerce invalid timestamps
    df = df.melt(id_vars="ts", var_name="car_id", value_name="value")
    df["car_id"] = df["car_id"].astype(str).str.strip().str.lower()
    df["metric"] = metric_name
    return df.dropna(subset=["value", "ts"])  # drop invalid rows

# --- Load and tag both datasets --- #
monthly_df = load_and_melt("monthly", "price")
annual_df = load_and_melt("annual", "popularity")

# --- Combine into one dataset --- #
combined_df = pd.concat([monthly_df, annual_df], ignore_index=True)

# --- Store in DuckDB --- #
con.execute("""
    CREATE TABLE IF NOT EXISTS car_price_popularity (
        car_id TEXT,
        ts TIMESTAMP,
        metric TEXT,
        value DOUBLE
    )
""")
con.register("combined_df", combined_df)
con.execute("INSERT INTO car_price_popularity SELECT * FROM combined_df")
con.execute("CREATE INDEX IF NOT EXISTS idx_car_price_popularity ON car_price_popularity(car_id, metric, ts)")

ConversionException: Conversion Error: invalid timestamp field format: "ferrari f40", expected format is (YYYY-MM-DD HH:MM:SS[.US][±HH[:MM[:SS]]| ZONE]) when casting from source column car_id

In [3]:
con.execute("DROP TABLE IF EXISTS car_price_popularity")

In [4]:
con.execute("""
CREATE TABLE car_price_popularity (
    car_id  TEXT,
    ts      TIMESTAMP,
    metric  TEXT,
    value   DOUBLE
)
""")

In [5]:
con.register("combined_df", combined_df)

con.execute("""
INSERT INTO car_price_popularity   -- target columns are fixed
        (car_id, ts, metric, value)
SELECT  car_id::TEXT,             -- make the casts explicit
        ts::TIMESTAMP,
        metric::TEXT,
        value::DOUBLE
FROM combined_df
""")

In [6]:
con.execute("SET GLOBAL pandas_analyze_sample=100000")  # or any big number

In [7]:
print(combined_df.dtypes)

ts        datetime64[ns]
car_id            object
value             object
metric            object
dtype: object


In [8]:
combined_df["value"] = pd.to_numeric(combined_df["value"], errors="coerce")
combined_df = combined_df.dropna(subset=["value", "ts"])

In [9]:
print(combined_df.dtypes)

ts        datetime64[ns]
car_id            object
value            float64
metric            object
dtype: object


In [10]:
combined_df = pd.concat([monthly_df, annual_df], ignore_index=True)

# 👉 Coerce numeric types
combined_df["value"] = pd.to_numeric(combined_df["value"], errors="coerce")
combined_df = combined_df.dropna(subset=["value", "ts"])

print(combined_df.dtypes)

ts        datetime64[ns]
car_id            object
value            float64
metric            object
dtype: object


In [11]:
con.register("combined_df", combined_df)

con.execute("""
INSERT INTO car_price_popularity (car_id, ts, metric, value)
SELECT 
    car_id::TEXT,
    ts::TIMESTAMP,
    metric::TEXT,
    value::DOUBLE
FROM combined_df
""")

In [12]:
con.execute("SELECT * FROM car_price_popularity LIMIT 5").fetchdf()

,car_id,ts,metric,value
0,ferrari f40,2006-01-01,price,231962.4644
1,ferrari f40,2006-02-01,price,242383.3204
2,ferrari f40,2006-03-01,price,252804.1765
3,ferrari f40,2006-04-01,price,263225.0325
4,ferrari f40,2006-05-01,price,273645.8886


In [13]:
schema_df = con.execute("DESCRIBE car_price_popularity").fetchdf()
print(schema_df)

  column_name column_type null   key default extra
0      car_id     VARCHAR  YES  None    None  None
1          ts   TIMESTAMP  YES  None    None  None
2      metric     VARCHAR  YES  None    None  None
3       value      DOUBLE  YES  None    None  None


In [14]:
schema_df = con.execute("DESCRIBE platform_metrics").fetchdf()
print(schema_df)

  column_name column_type null   key default extra
0      car_id     VARCHAR   NO   PRI    None  None
1    platform     VARCHAR   NO   PRI    None  None
2      run_ts   TIMESTAMP   NO   PRI    None  None
3     metrics        JSON  YES  None    None  None


In [17]:
schema_df = con.execute("DESCRIBE overall_cache").fetchdf()
print(schema_df)

  column_name column_type null   key default extra
0      car_id     VARCHAR   NO   PRI    None  None
1      run_ts   TIMESTAMP   NO   PRI    None  None
2     metrics        JSON  YES  None    None  None


In [19]:
import duckdb

with duckdb.connect(db_path) as con:
    row = con.execute("""
        SELECT *
        FROM overall_cache
        USING SAMPLE 1
    """).fetchone()

print(row)

None


In [16]:
import json
from datetime import datetime
from vintage_ai.api.core.schemas.v1 import CarMetric


def aggregate_metrics_and_cache(car_id: str) -> list[CarMetric]:

    # Step 1: fetch latest metrics per platform
    rows = con.execute("""
        SELECT platform, metrics
        FROM (
            SELECT *, row_number() OVER (PARTITION BY platform ORDER BY run_ts DESC) AS rn
            FROM platform_metrics
            WHERE car_id = ?
        )
        WHERE rn = 1
    """, [car_id]).fetchall()

    # Step 2: merge all JSON blobs
    merged: dict[str, float | int | None] = {}
    for _platform, metrics_json in rows:
        merged |= json.loads(metrics_json)

    # Step 3: enrich with price/popularity trend if present
    price_rows = con.execute("""
        SELECT metric, value
        FROM car_price_popularity
        WHERE car_id = ?
          AND ts >= NOW() - INTERVAL 1 YEAR
    """, [car_id]).fetchall()

    # Collect the most recent value for each metric
    recent = {}
    for metric, value in price_rows:
        recent[metric] = value  # will be overwritten with last (latest) row

    # Add to merged result
    merged.update({
        f"latest_{k}": v for k, v in recent.items()
    })

    # Step 4: cache the result
    con.execute("""
        INSERT OR REPLACE INTO overall_cache
        VALUES (?, ?, ?)
    """, [car_id, datetime.utcnow(), json.dumps(merged)])

    # Step 5: return as list[CarMetric]
    return [CarMetric(metric=k, value=v) for k, v in merged.items()]

ModuleNotFoundError: No module named 'vintage_ai'